# Building a Regression MLP Using the Sequential API
Let’s switch to the California housing problem and tackle it using a
regression neural network.   
After loading the data, we split it into a training set, a
validation set, and a test set, and we scale all the features:

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
X_train_full, y_train_full)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [3]:
model = keras.models.Sequential([
keras.layers.Dense(30, activation="relu",
input_shape=X_train.shape[1:]),
keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20,
validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model.predict(X_new)

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 70us/sample - loss: 1.0116 - val_loss: 0.6095
Epoch 2/20
11610/11610 [==============================] - ETA: 0s - loss: 0.484 - 1s 44us/sample - loss: 0.4834 - val_loss: 0.4440
Epoch 3/20
11610/11610 [==============================] - 0s 38us/sample - loss: 0.4383 - val_loss: 0.4211
Epoch 4/20
11610/11610 [==============================] - 0s 41us/sample - loss: 0.4239 - val_loss: 0.4251
Epoch 5/20
11610/11610 [==============================] - 1s 46us/sample - loss: 0.4123 - val_loss: 0.4100
Epoch 6/20
11610/11610 [==============================] - 1s 43us/sample - loss: 0.4030 - val_loss: 0.3933
Epoch 7/20
11610/11610 [==============================] - 0s 43us/sample - loss: 0.3973 - val_loss: 0.3829
Epoch 8/20
11610/11610 [==============================] - 1s 48us/sample - loss: 0.3947 - val_loss: 0.3785
Epoch 9/20
11610/11610 [==============================] - 0s 41us/sampl

As you can see, the Sequential API is quite easy to use. However, although
Sequential models are extremely common, it is sometimes useful to
build neural networks with more complex topologies, or with multiple
inputs or outputs. For this purpose, Keras offers the Functional API.


# Building Complex Models Using the Functional API
One example of a nonsequential neural network is a Wide & Deep neural
network. This neural network architecture was introduced in a 2016 paper
by Heng-Tze Cheng et al. It connects all or part of the inputs directly to
the output layer,

In [4]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

Let’s go through each line of this code:
* First, we need to create an Input object. This is a specification
of the kind of input the model will get, including its shape and
dtype. A model may actually have multiple inputs, as we will see
shortly.
* Next, we create a Dense layer with 30 neurons, using the ReLU
activation function. As soon as it is created, notice that we call it
like a function, passing it the input. This is why this is called the
Functional API. Note that we are just telling Keras how it should
connect the layers together; no actual data is being processed yet.
* We then create a second hidden layer, and again we use it as a
function. Note that we pass it the output of the first hidden layer.
* Next, we create a Concatenate layer, and once again we
immediately use it like a function, to concatenate the input and
the output of the second hidden layer. You may prefer the
keras.layers.concatenate() function, which creates a
Concatenate layer and immediately calls it with the given inputs.
* Then we create the output layer, with a single neuron and no
activation function, and we call it like a function, passing it the
result of the concatenation.

Lastly, we create a Keras Model, specifying which inputs and
outputs to use.  
If you want to send a subset of the features through the wide path
and a different subset (possibly overlapping) through the deep path? In this case, one solution is to use multiple inputs. For
example, suppose we want to send five features through the wide path
(features 0 to 4), and six features through the deep path (features 2 to 7)


In [5]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

In [6]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]
history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))


Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 66us/sample - loss: 2.0090 - val_loss: 0.9477
Epoch 2/20
11610/11610 [==============================] - 0s 41us/sample - loss: 0.7577 - val_loss: 0.7546
Epoch 3/20
11610/11610 [==============================] - 0s 42us/sample - loss: 0.6824 - val_loss: 0.6970
Epoch 4/20
11610/11610 [==============================] - 0s 40us/sample - loss: 0.6386 - val_loss: 0.6536
Epoch 5/20
11610/11610 [==============================] - 0s 40us/sample - loss: 0.6023 - val_loss: 0.6145
Epoch 6/20
11610/11610 [==============================] - 0s 41us/sample - loss: 0.5726 - val_loss: 0.5806
Epoch 7/20
11610/11610 [==============================] - 0s 41us/sample - loss: 0.5463 - val_loss: 0.5525
Epoch 8/20
11610/11610 [==============================] - 0s 43us/sample - loss: 0.5261 - val_loss: 0.5316
Epoch 9/20
11610/11610 [==============================] - 0s 40us/sample - loss: 0.5106 - val_l